<a href="https://colab.research.google.com/github/emoceanographer/usdanutrients/blob/master/Protein_requirements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import re
import statistics
from operator import add
import numpy as np

import math
import scipy
from scipy import stats
from scipy.optimize import least_squares

In [0]:
path = '/content/drive/My Drive/Colab Notebooks/Nutrition/'
pop_path = path + 'WPP2019_PopulationByAgeSex_Medium.csv'
#BMI_path = path + 'NCD_BMI_MEAN.csv'
BMI_path = path + 'NCD_RisC_Lancet_2017_BMI_age_standardised_country.csv'
height_path = path + 'heights_over_time.csv'
fao_ctry_path = path + 'fao_ctry_codes.csv'
CV_path = path + 'Food_Security_Indicators.xlsx'

# Calculate avg. body mass of adults for each country

Population: Dataset is WPP2019, Population by age and sex, medium; for each country; plan is to, for each (country, year), match demographic groups to the age_aa requirement pattern and calculate needed aa requirements

In [0]:
pop_df = pd.read_csv(pop_path, encoding='latin-1') # each country has year, age group's population
BMI_df = pd.read_csv(BMI_path, encoding='latin-1') # each country has a year and an associated mean
height_df = pd.read_csv(height_path)
fao_ctry_codes = pd.read_csv(fao_ctry_path)

In [16]:
BMI_df.head()

,Country/Region/World,ISO,Sex,Year,Mean BMI,Mean BMI lower 95% uncertainty interval,Mean BMI upper 95% uncertainty interval,Prevalence of BMI>=30 kg/m² (obesity),Prevalence of BMI>=30 kg/m² lower 95% uncertainty interval,Prevalence of BMI>=30 kg/m² upper 95% uncertainty interval,Prevalence of BMI>=35 kg/m² (severe obesity),Prevalence of BMI>=35 kg/m² lower 95% uncertainty interval,Prevalence of BMI>=35 kg/m² upper 95% uncertainty interval,Prevalence of BMI<18.5 kg/m² (underweight),Prevalence of BMI<18.5 kg/m² lower 95% uncertainty interval,Prevalence of BMI<18.5 kg/m² upper 95% uncertainty interval,Prevalence of BMI 18.5 kg/m² to <20 kg/m²,Prevalence of BMI 18.5 kg/m² to <20 kg/m² lower 95% uncertainty interval,Prevalence of BMI 18.5 kg/m² to <20 kg/m² upper 95% uncertainty interval,Prevalence of BMI 20 kg/m² to <25 kg/m²,Prevalence of BMI 20 kg/m² to <25 kg/m² lower 95% uncertainty interval,Prevalence of BMI 20 kg/m² to <25 kg/m² upper 95% uncertainty interval,Prevalence of BMI 25 kg/m² to <30 kg/m²,Prevalence of BMI 25 kg/m² to <30 kg/m² lower 95% uncertainty interval,Prevalence of BMI 25 kg/m² to <30 kg/m² upper 95% uncertainty interval,Prevalence of BMI 30 kg/m² to <35 kg/m²,Prevalence of BMI 30 kg/m² to <35 kg/m² lower 95% uncertainty interval,Prevalence of BMI 30 kg/m² to <35 kg/m² upper 95% uncertainty interval,Prevalence of BMI 35 kg/m² to <40 kg/m²,Prevalence of BMI 35 kg/m² to <40 kg/m² lower 95% uncertainty interval,Prevalence of BMI 35 kg/m² to <40 kg/m² upper 95% uncertainty interval,Prevalence of BMI >=40 kg/m²(morbid obesity),Prevalence of BMI >=40 kg/m² lower 95% uncertainty interval,Prevalence of BMI >=40 kg/m² upper 95% uncertainty interval
0,Afghanistan,AFG,Men,1975,18.999440,16.331119,21.763077,0.002078,0.000384,0.006515,0.000034,0.000002,0.000158,0.284239,0.147565,0.421922,0.234271,0.134912,0.347236,0.428146,0.312880,0.551022,0.051266,0.021452,0.096419,0.002043,0.000351,0.006493,0.000027,6.663263e-07,0.000138,0.000008,5.054664e-08,0.000047
1,Afghanistan,AFG,Men,1976,19.105518,16.500077,21.801109,0.002237,0.000437,0.006820,0.000038,0.000002,0.000168,0.281990,0.148634,0.417047,0.232208,0.136020,0.340563,0.430523,0.317865,0.549570,0.053042,0.022928,0.097881,0.002199,0.000403,0.006779,0.000030,8.525116e-07,0.000149,0.000009,6.816061e-08,0.000049
2,Afghanistan,AFG,Men,1977,19.212080,16.646507,21.842021,0.002410,0.000497,0.007179,0.000042,0.000003,0.000185,0.279694,0.149224,0.412887,0.230125,0.136671,0.335139,0.432881,0.322810,0.549225,0.054890,0.024613,0.098869,0.002368,0.000454,0.007144,0.000033,1.080451e-06,0.000163,0.000009,9.255251e-08,0.000053
3,Afghanistan,AFG,Men,1978,19.319053,16.786899,21.876516,0.002599,0.000559,0.007567,0.000047,0.000003,0.000202,0.277336,0.150410,0.407314,0.228031,0.137231,0.330865,0.435219,0.327795,0.548325,0.056815,0.026318,0.100453,0.002551,0.000519,0.007540,0.000037,1.364412e-06,0.000178,0.000010,1.262377e-07,0.000057
4,Afghanistan,AFG,Men,1979,19.425952,16.950002,21.930634,0.002804,0.000636,0.008026,0.000053,0.000004,0.000220,0.274931,0.151589,0.403758,0.225927,0.137064,0.327193,0.437522,0.332613,0.547925,0.058816,0.028116,0.102426,0.002751,0.000581,0.007971,0.000041,1.698723e-06,0.000193,0.000011,1.576738e-07,0.000062


In [0]:
def BMI_extract(df,yrs):
  """Gets the average and standard deviation for selected years for male and female BMI"""
  # Male
  temp = df[(df['Sex']=='Men') & (df['Year'].isin(yrs))]
  avgM = temp['Mean BMI'].mean(axis=0)
  #stdM = ((temp['Mean BMI upper 95% uncertainty interval'] - temp['Mean BMI lower 95% uncertainty interval'])/4).mean(axis=0)
  
   # Female
  temp = df[(df['Sex']=='Women') & (df['Year'].isin(yrs))]
  avgF = temp['Mean BMI'].mean(axis=0)
  #stdF = ((temp['Mean BMI upper 95% uncertainty interval'] - temp['Mean BMI lower 95% uncertainty interval'])/4).mean(axis=0)
  
  return avgM, avgF

In [0]:
def perc_calc(percentile, mean_val, std):
  """For a normal distribution, calculate the value of a normal distribution at the nth percentile"""
  x = scipy.special.erfinv(2*percentile-1)*std*2**.5 + mean_val
  return x

In [0]:
# Age group definitions
child_age_grps = ['0-4', '5-9', '10-14', '15-19']
adult_age_grps = ['20-24', '25-29', '30-34', '35-39',
       '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74',
       '75-79', '80-84', '85-89', '90-94', '95-99', '100+'] # age groups for pop. data

In [0]:
ctry_mismatches = {'China, Hong Kong SAR': ['none'], 'China, Macao SAR': ['none'], 
                   'China, Taiwan Province of China': ['none'], 
                   "Dem. People's Republic of Korea": "Democratic People's Republic of Korea",
                   'French Guiana': ['none'], 'French Polynesia': ['none'], 
                   'Guadeloupe': ['none'],'Martinique': ['none'],
                   'United Kingdom': ['United Kingdom of Great Britain and Northern Ireland'],
                   'Puerto Rico': ['none']}

In [10]:
# for each year and country, calculate the average mass of an adult
avg_weight = {}
avg_underweight = {}


for country in list(pop_df.Location.unique()): # cycle through each country
  # population
  temp = pop_df[pop_df['Location'].isin([country])]
  temp = temp[temp['AgeGrp'].isin(adult_age_grps)]
  
  # BMI
  ctr = 0
  if country in BMI_df['Country/Region/World'].unique(): # skip countries for which there is no BMI
    ctr = 1
    BMI = BMI_df[BMI_df['Country/Region/World'] == country]
    
  if country in ctry_mismatches: 
    if ctry_mismatches[country][0] in BMI_df['Country/Region/World'].unique():
      BMI = BMI_df[BMI_df['Country/Region/World'] == ctry_mismatches[country][0]]
      print('mismatch')
    if ctry_mismatches[country] == ['none']:
        ctr = 0
  if ctr == 1:

    if country == 'United Kingdom':
      ctry_code = 'GBR'
    else:
      ctry_code = fao_ctry_codes[fao_ctry_codes['Short name'] == country]['ISO3'].values[0]
    
    # Heights (gets people 20-40 in 2011-2013 by year of birth)
    hts = height_df[height_df['Code'] == ctry_code]
    hts_m = hts[(hts['Year']>1970) & (hts['Year']<1994)]['Males (centimeters)'].mean()
    hts_f = hts[(hts['Year']>1970) & (hts['Year']<1994)]['Females (centimeters)'].mean()
    
    f_pop_year = []
    m_pop_year = []
    f_BMI = []
    m_BMI = []
    
    for year in list(pop_df.Time.unique()):
      if year < 2014 and year > 2010: # get 2011-2013
        # population
        f_adult_yr_pop = sum(temp[temp['Time'].isin([year])]['PopFemale'])
        m_adult_yr_pop = sum(temp[temp['Time'].isin([year])]['PopMale'])
        f_pop_year.append(f_adult_yr_pop*1000) # units originally in thousands of people
        m_pop_year.append(m_adult_yr_pop*1000)

        # BMI (average only)
    meanM,meanF = BMI_extract(BMI,[2011,2012,2013])

    f_uw =  (hts_f/100)**2 * 18.5
    m_uw = (hts_m/100)**2 * 18.5
    total_underweight = (f_uw*statistics.mean(f_pop_year) + m_uw*statistics.mean(m_pop_year)) / statistics.mean(list(map(add, f_pop_year, m_pop_year)))
    
    p_muw = BMI[(BMI['Sex']=='Men') & (BMI['Year'].isin([2011,2012,2013]))]['Prevalence of BMI<18.5 kg/m² (underweight)'].values[0].mean()
    p_fuw = BMI[(BMI['Sex']=='Women') & (BMI['Year'].isin([2011,2012,2013]))]['Prevalence of BMI<18.5 kg/m² (underweight)'].values[0].mean()
    
    # Weight calculation (kg)
    f_weight = (hts_f/100)**2 * meanF
    m_weight = (hts_m/100)**2 * meanM
    
    # Population weighting
    avg_weight[country] = (f_weight*statistics.mean(f_pop_year) + m_weight*statistics.mean(m_pop_year)) / statistics.mean(list(map(add, f_pop_year, m_pop_year)))
    avg_underweight[country] = {'weight': total_underweight, 'percent': (p_fuw*statistics.mean(f_pop_year) + p_muw*statistics.mean(m_pop_year)) / statistics.mean(list(map(add, f_pop_year, m_pop_year)))}


  else:
    print(country)

Africa
Aruba
Asia
Australia/New Zealand
Bolivia (Plurinational State of)
Caribbean
Central America
Central and Southern Asia
Central Asia
Channel Islands
China, Hong Kong SAR
China, Macao SAR
China, Taiwan Province of China
Côte d'Ivoire
Curaçao
Czechia
Dem. People's Republic of Korea
Democratic Republic of the Congo
Eastern Africa
Eastern and South-Eastern Asia
Eastern Asia
Eastern Europe
Eswatini
Europe
Europe and Northern America
French Guiana
French Polynesia
Guadeloupe
Guam
Guinea-Bissau
High-income countries
Iran (Islamic Republic of)
Land-locked Developing Countries (LLDC)
Lao People's Democratic Republic
Latin America and the Caribbean
Least developed countries
Less developed regions
Less developed regions, excluding China
Less developed regions, excluding least developed countries
Low-income countries
Lower-middle-income countries
Martinique
Mayotte
Melanesia
Micronesia
Micronesia (Fed. States of)
Middle Africa
Middle-income countries
More developed regions
New Caledonia
No in

In [11]:
# Selects the average UNDERWEIGHT amount (corresponding to BMI of 18.5)
ctry_weights = pd.DataFrame.from_dict(avg_underweight,orient='index')
ctry_weights = ctry_weights.reset_index()
missing = ctry_weights[ctry_weights['weight'].isna()]
list(missing['index'].unique()) # should be just those with actually no data (e.g. Sudan)

[]

# Calculate AA requirements per capita (adults)

In [0]:
case = 'safe'

if case == 'baseline':

  AA_adult_req = {'Histidine': 15, 'Isoleucine': 30, 'Leucine': 59, 'Lysine': 45, 
                 'SAA': 22, 'AAA': 38, 'Threonine': 23, 'Tryptophan': 6, 'Valine': 39} # mg AA /g protein
  AA_total = .66 # g/kg
elif case == 'safe':
  AA_adult_req = {'Histidine': 12, 'Isoleucine': 24, 'Leucine': 47, 'Lysine': 36, 
                 'SAA': 17, 'AAA': 30, 'Threonine': 18, 'Tryptophan': 5, 'Valine': 31} # mg AA /g protein
  AA_total = .83 # g/kg

In [0]:
SAA = ['Methionine', 'Cystine'] # there are others, but not in my diet AA list
AAA = ['Phenylalanine', 'Tyrosine'] # also tryptophan and histidine but they are listed separately

In [0]:
# NOT EAT-Lancet
diet_AA = pd.read_csv(path + 'diet_aa_1126_waste.csv')

In [0]:
# EAT-Lancet
diet_AA = pd.read_csv(path + 'diet_aa_eat_lancet.csv')


In [113]:
diet_AA.head()

,Unnamed: 0,Tryptophan,Threonine,Isoleucine,Leucine,Lysine,Methionine,Cystine,Phenylalanine,Tyrosine,Valine,Arginine,Histidine,Alanine,Aspartic,Glutamic,Glycine,Serine,protein
0,all-r,0.812312,2.809532,3.195671,5.628226,5.337084,1.738398,0.868954,3.316345,2.364952,3.545661,4.869229,1.974018,1.555892,3.639414,7.938239,1.261111,1.649401,75.039876
1,HIC,1.045144,3.688985,4.152630,7.267671,7.114227,2.301135,1.115869,4.223912,3.058003,4.574413,6.016159,2.567456,1.701531,3.890532,9.281779,1.409469,1.891238,98.039272
2,UMC,1.010827,3.586597,4.045459,7.124699,6.895349,2.243851,1.085585,4.128442,2.991701,4.466618,5.863973,2.494308,1.675468,3.809896,8.958443,1.342276,1.841170,95.148158
3,LMC,0.772347,2.656309,3.030521,5.330312,5.023109,1.641076,0.825398,3.161281,2.235544,3.369052,4.685313,1.865697,1.530336,3.620562,7.735272,1.241755,1.605787,71.038289
4,LIC,0.541086,1.764498,2.050614,3.686786,3.252588,1.059165,0.579840,2.219690,1.565047,2.305024,3.459683,1.292176,1.394951,3.306395,6.435862,1.108182,1.396608,47.978510


In [0]:
names = {'Cape Verde': 'Cabo Verde', 'Czech Republic': 'Czechia', 
         'C√¥te d&#39;Ivoire': "Côte d'Ivoire", 'Côte d&#39;Ivoire': "Côte d'Ivoire",
         'Democratic People&#39;s Republic of Korea':
        "Democratic People's Republic of Korea", 
         'Faroe Islands (Associate Member)': 'Faroe Islands', 
         'Lao People&#39;s Democratic Republic': "Lao People's Democratic Republic",
          'Swaziland': 'Eswatini', 'Tokelau (Associate Member)': 'Tokelau',
        'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland'}
ignore = ['Holy See','Liechtenstein','The former Yugoslav Republic of Macedonia',
          np.nan,'Antarctica']

In [0]:
def name2iso3(name, names, ctry_codes):
  if name in names:
    iso3 = ctry_codes[ctry_codes['Short name'] == names[name]]['ISO3'].values[0]
  elif name in ignore:
    iso3 = np.nan
  else:
    iso3 = ctry_codes[ctry_codes['Short name'] == name]['ISO3'].values[0]
  return iso3

In [0]:
isos = []
for country in ctry_weights['index'].unique(): # assigns ISO3 values to country names
  isos.append(name2iso3(country,names,fao_ctry_codes))
ctry_weights['ISO3'] = isos

In [18]:
# EAT-Lancet version
AA_ctry = {}
AA_all = {}
for country in diet_AA['Unnamed: 0'].unique().tolist():
  if country in ctry_weights['ISO3'].unique().tolist():
    diet_ctry = diet_AA[diet_AA['Unnamed: 0'] == country]
    AA_ctry[country] = {}
    AA_all[country] = {}
    for AA in AA_adult_req: 
      avg_weight = ctry_weights[ctry_weights['ISO3']==country]['weight'].values[0] 
      AA_req =  avg_weight * AA_total * AA_adult_req[AA] * 0.001 #g/capita/day
      if AA == 'SAA':
        aa_diet = []
        for aa in SAA: 
          aa_diet.append(diet_ctry[aa].values[0])
        AA_diet = sum(aa_diet)
      elif AA == 'AAA':
        aa_diet = []
        for aa in AAA: 
          aa_diet.append(diet_ctry[aa].values[0])
        AA_diet = sum(aa_diet)
      else:
        AA_diet = diet_ctry[AA].values[0]
      AA_ctry[country][AA] = AA_diet / AA_req #[AA_req, AA_diet, AA_diet/AA_req] 
      AA_all[country][AA] = [AA_req, AA_diet, AA_diet/AA_req] 
      if AA_ctry[country][AA]<1:
        print(country, ' lacking ' , AA, ' see: ', AA_req, AA_diet)
    total_protein_req = avg_weight * AA_total
    total_protein_diet = diet_ctry['protein'].values[0]
    AA_ctry[country]['protein'] =  total_protein_diet/total_protein_req #[total_protein_req, total_protein_diet, total_protein_diet/total_protein_req]
    AA_all[country]['protein'] = [total_protein_req, total_protein_diet, total_protein_diet/total_protein_req]
    if AA_ctry[country]['protein'] < 1: 
      print(country, ' lacking total protein, see' , total_protein_req, total_protein_diet)

BDI  lacking total protein, see 40.22304740634166 35.03202221864809
CAF  lacking total protein, see 41.17876392106861 37.22976665257153
ERI  lacking total protein, see 41.031221395992034 36.90101942190607
ETH  lacking total protein, see 40.44180132603546 38.69235505439807
GIN  lacking total protein, see 41.28102389063809 36.8499990610025
MOZ  lacking total protein, see 39.53746115962086 39.439849023669204
MWI  lacking total protein, see 39.26671319363268 37.40819184362724
NER  lacking total protein, see 41.89221865795627 41.535350745583365
RWA  lacking total protein, see 39.775335029288 33.38673352125982
SDN  lacking total protein, see 40.59793311283723 38.28476410434689
TCD  lacking total protein, see 42.93093663498566 37.52971778859633
GNQ  lacking total protein, see 41.73330701861095 27.76491139712421


In [19]:
ctry_weights.head()

,index,weight,percent,ISO3
0,Afghanistan,47.817386,0.179144,AFG
1,Albania,52.225617,0.013541,ALB
2,Algeria,50.823507,0.038531,DZA
3,Angola,49.653432,0.143579,AGO
4,Antigua and Barbuda,51.204092,0.048547,ATG


In [96]:
AA_ctry = {}
AA_all = {}
for country in diet_AA['Unnamed: 0'].unique().tolist():
  if country in ctry_weights['index'].unique().tolist():
    diet_ctry = diet_AA[diet_AA['Unnamed: 0'] == country]
    AA_ctry[country] = {}
    AA_all[country] = {}
    for AA in AA_adult_req: 
      avg_weight = ctry_weights[ctry_weights['index']==country]['weight'].values[0] 
      AA_req =  avg_weight * AA_total * AA_adult_req[AA] * 0.001 #g/capita/day
      if AA == 'SAA':
        aa_diet = []
        for aa in SAA: 
          aa_diet.append(diet_ctry[aa].values[0])
        AA_diet = sum(aa_diet)
      elif AA == 'AAA':
        aa_diet = []
        for aa in AAA: 
          aa_diet.append(diet_ctry[aa].values[0])
        AA_diet = sum(aa_diet)
      else:
        AA_diet = diet_ctry[AA].values[0]
      AA_ctry[country][AA] = AA_diet / AA_req #[AA_req, AA_diet, AA_diet/AA_req] 
      AA_all[country][AA] = [AA_req, AA_diet, AA_diet/AA_req] 
      if AA_ctry[country][AA]<1:
        print(country, ' lacking ' , AA, ' see: ', AA_req, AA_diet)
    total_protein_req = avg_weight * AA_total
    total_protein_diet = diet_ctry['protein'].values[0]
    AA_ctry[country]['protein'] =  total_protein_diet/total_protein_req #[total_protein_req, total_protein_diet, total_protein_diet/total_protein_req]
    AA_all[country]['protein'] = [total_protein_req, total_protein_diet, total_protein_diet/total_protein_req]
    if AA_ctry[country]['protein'] < 1: 
      print(country, ' lacking total protein, see' , total_protein_req, total_protein_diet)

Bangladesh  lacking total protein, see 38.215221558512525 31.73755243612161
Benin  lacking total protein, see 41.308831286607884 35.81830660081956
Cameroon  lacking total protein, see 41.965491167236785 34.05241593091996
Central African Republic  lacking total protein, see 41.17876392106861 33.65412431702833
Chad  lacking total protein, see 42.93093663498566 36.96810560982088
Congo  lacking total protein, see 41.338114340600804 39.85403459190722
Ethiopia  lacking total protein, see 40.44180132603546 29.94893009926227
Ghana  lacking total protein, see 41.63675072023671 33.61477120774747
Guinea  lacking total protein, see 41.28102389063809 29.39187268097221
Haiti  lacking  Valine  see:  1.306598661793003 1.2455306182289656
Haiti  lacking total protein, see 42.14834392880655 25.433505204294267
India  lacking total protein, see 38.84241061649938 36.577197702064936
Kenya  lacking total protein, see 42.011657259391505 38.443763311597685
Lesotho  lacking total protein, see 40.41992514950422 4

In [0]:
AA_ratios = pd.DataFrame.from_dict(AA_ctry,orient='index')

In [123]:
AA_ratios.head()

,Histidine,Isoleucine,Leucine,Lysine,SAA,AAA,Threonine,Tryptophan,Valine,protein
AFG,2.905914,2.277211,2.085556,2.312981,2.576473,3.407359,2.562864,3.026305,1.988301,1.359870
AGO,4.163365,3.193596,2.855765,3.796484,3.763811,4.517659,3.830509,3.972618,2.676474,1.808069
ALB,4.772125,3.879717,3.482604,4.221056,4.541501,5.509501,4.511079,4.793023,3.342097,2.290365
ARG,5.224301,4.222130,3.790259,4.695960,4.961619,5.967132,4.950581,5.167247,3.618682,2.465254
ARM,4.242898,3.497986,3.153597,3.710548,4.033916,5.011783,4.015676,4.323493,3.033893,2.071745


In [0]:
savepath = path + 'AA_ratios_EATLanceteven_safe_1217.csv'
AA_ratios.to_csv(savepath)

## Undernourishment
Use the average diet as the mean of lognormal & the avg. requirement as threshold for undernourishment (NB FAO would use a 5th percentile for this?)

In [0]:
CVs = pd.read_excel(CV_path,sheet_name='A_6',header=2)

In [49]:
CVs.head()

,FAOST_CODE,Regions/Subregions/Countries,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,5001,World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5852,Developing countries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5100,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,421,Northern Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Algeria,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.27,0.27,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28


In [0]:
def prop_under(s_mean, CV, x):
  sigma = ((math.log(CV**2+1)))**0.5
  mu = math.log(s_mean) - sigma**2/2
  CDF = 1/2 + 1/2*scipy.special.erf((math.log(x) - mu)/(2**0.5*sigma))
  return CDF

In [0]:
def CV_calc(df,years):
  """Expects dataframe for a country and will extract coefficient of variation
  for the selected years"""
  vals = df[years].values[0]
  mean_val = statistics.mean(vals)
  if np.isnan(mean_val):
    mean_val = 0.2 # conservative coefficient of variation if no data
  return mean_val

In [125]:
# calculates the proportion under each
variable_diet = {}
for country in AA_all:
  variable_diet[country] = {}
  ctry_CV = CVs[CVs['Regions/Subregions/Countries']==country]
  CV = CV_calc(ctry_CV,[2011,2012,2013])
  #print([country,CV])
  for AA in AA_all[country]:
    pr_mean = AA_all[country][AA][1] # average amount in diet
    req = AA_all[country][AA][0] # average requirement

    CDF = prop_under(pr_mean, CV, req)
    variable_diet[country][AA] = CDF

IndexError: ignored

In [47]:
CVs.head()

,FAOST_CODE,Regions/Subregions/Countries,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014
0,5001,World,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5852,Developing countries,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5100,Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,421,Northern Africa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Algeria,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.27,0.27,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28


In [52]:
fao_ctry_codes.head()

,Short name,Official name,ISO3,ISO2,UNI,UNDP,FAOSTAT,GAUL
0,Afghanistan,the Islamic Republic of Afghanistan,AFG,AF,4,AFG,2,1
1,Albania,the Republic of Albania,ALB,AL,8,ALB,3,3
2,Algeria,the People's Democratic Republic of Algeria,DZA,DZ,12,DZA,4,4
3,Andorra,the Principality of Andorra,AND,AD,20,AND,6,7
4,Angola,the Republic of Angola,AGO,AO,24,AGO,7,8


In [0]:
def fao2iso3(code, fao_ctry_codes):
  """Gets iso3 from fao code"""
  if code in fao_ctry_codes['FAOSTAT'].unique():
    iso3 = list(fao_ctry_codes[fao_ctry_codes['FAOSTAT']==code]['ISO3'])[0]
  else:
    iso3 = np.nan
  return iso3

In [0]:
isos = []
for faocode in CVs['FAOST_CODE'].unique():
  isos.append(fao2iso3(faocode,fao_ctry_codes))
CVs['ISO3'] = isos

In [0]:
# EAT-Lancet version; calculates the proportion under each
variable_diet = {}
for country in AA_all:
  variable_diet[country] = {}
  ctry_CV = CVs[CVs['ISO3']==country]
  CV = CV_calc(ctry_CV,[2011,2012,2013])
  #print([country,CV])
  for AA in AA_all[country]:
    pr_mean = AA_all[country][AA][1] # average amount in diet
    req = AA_all[country][AA][0] # average requirement

    CDF = prop_under(pr_mean, CV, req)
    variable_diet[country][AA] = CDF


In [60]:
variable_diet_df = pd.DataFrame.from_dict(variable_diet,orient='index')
variable_diet_df.head()

,Histidine,Isoleucine,Leucine,Lysine,SAA,AAA,Threonine,Tryptophan,Valine,protein
AFG,1.280379e-05,6.426930e-04,2.103820e-03,5.144144e-04,9.934244e-05,5.991797e-07,1.081512e-04,6.092600e-06,3.812835e-03,0.130227
AGO,1.226930e-06,6.987344e-05,3.043057e-04,5.470536e-06,6.262490e-06,3.016896e-07,4.753786e-06,2.654779e-06,6.709270e-04,0.030692
ALB,3.275158e-15,7.556844e-12,2.795377e-10,3.679834e-13,2.314815e-14,0.000000e+00,2.997602e-14,2.775558e-15,1.026152e-09,0.000022
ARG,1.998401e-15,4.258816e-12,1.394005e-10,1.055822e-13,1.409983e-14,0.000000e+00,1.532108e-14,3.053113e-15,5.747448e-10,0.000011
ARM,1.056731e-09,1.151641e-07,1.094930e-06,2.941633e-08,3.838448e-09,1.096156e-11,4.300098e-09,6.461424e-10,2.420744e-06,0.001540


In [0]:
savepath = path + 'undernourished_CVs_EL_1218.csv'
variable_diet_df.to_csv(savepath)

# Old Code

In [0]:
# BMI regex work! (save me for now!)
temp = BMI['2016'].values[0]
print(temp)
m = re.search(r"[^[]*\[([^]]*)\]", temp).groups()[0] # gets the things in the brackets!
n = re.search(r'^.*(?=(\[))', temp).group(0)
print(m)
print(n)

23.4 [22.0-24.8]
22.0-24.8
23.4 


In [0]:
adult_weight = 136.68 # average adult weight


In [0]:
# calculate average height (m) for each country; look into this later
ctry_hts = {}
for country in list(pop_df.Location.unique()): 
  if country in list(height_df.Entity.unique()):
    ht = height_df[height_df['Entity'].isin([country])]
    m_ht = statistics.mean(ht['Males (centimeters)'])
    f_ht = statistics.mean(ht['Females (centimeters)'])
    ctry_hts[country] = {'M': m_ht/100, 'F': f_ht/100}

In [0]:
# match those historical populations with average weight
avg_adult_weight = {}
total_adult_weight = {}

for country in list(pop_df.Location.unique()): 
  BMI = BMI_df[BMI_df['Country'].isin([country])]
  f_yrs_weight = {}
  m_yrs_weight = {}
  
  ctry_ht = ctry_hts.get(country)
  if ctry_ht:
    m_ht = ctry_ht['M'] # get the average height for that country
    f_ht = ctry_ht['F']

    f_tot_weight = {} # pre-allocate
    m_tot_weight = {}
    tot_yr_weight = {}
    for year in list(pop_df.Time.unique()):
      # BMI
      if str(year) in BMI.columns: # only go through years for which we have data
        avg_m_BMI = re.sub(r'\[[^]]*\]','',BMI[str(year)+'.1'].item())
        avg_f_BMI = re.sub(r'\[[^]]*\]','',BMI[str(year)+'.2'].item())
        
        f_yrs_weight[year] = float(avg_f_BMI)*(f_ht)**2 # calculates weight from BMI and height
        m_yrs_weight[year] = float(avg_m_BMI)*(m_ht)**2
        
        f_tot_weight[year] = f_adult_population[country][year]*f_yrs_weight[year]
        m_tot_weight[year] = m_adult_population[country][year]*m_yrs_weight[year]
        tot_yr_weight[year] = f_tot_weight[year] + m_tot_weight[year]
        
    avg_adult_weight[country] = {'f': f_yrs_weight, 'm': m_yrs_weight}
  
    total_adult_weight[country] = tot_yr_weight
  else:
    print([country, ' did not have associated height'])

['Africa', ' did not have associated height']
['Aruba', ' did not have associated height']
['Asia', ' did not have associated height']
['Australia/New Zealand', ' did not have associated height']
['Bolivia (Plurinational State of)', ' did not have associated height']
['Brunei Darussalam', ' did not have associated height']
['Cabo Verde', ' did not have associated height']
['Caribbean', ' did not have associated height']
['Central America', ' did not have associated height']
['Central and Southern Asia', ' did not have associated height']
['Central Asia', ' did not have associated height']
['Channel Islands', ' did not have associated height']
['China, Hong Kong SAR', ' did not have associated height']
['China, Macao SAR', ' did not have associated height']
['China, Taiwan Province of China', ' did not have associated height']
["Côte d'Ivoire", ' did not have associated height']
['Curaçao', ' did not have associated height']
['Czechia', ' did not have associated height']
["Dem. People's

ValueError: ignored